In [1]:
from pydub import AudioSegment

In [2]:
# Load your audio file
audio = AudioSegment.from_file("./audio/bal_train/--cB2ZVjpnA.flac")

# Get sampling rate
sampling_rate = audio.frame_rate

# Get bitrate
bitrate = audio.frame_width * audio.frame_rate * 8

print(f"Frame Width: {audio.frame_width} (I think it's in Bytes)")
print(f"Sampling Rate: {sampling_rate/1000} kHz")
print(f"Bitrate: {bitrate/1000} kbps")

Frame Width: 8 (I think it's in Bytes)
Sampling Rate: 48.0 kHz
Bitrate: 3072.0 kbps


## Resample sanity check

In [3]:
# Load the FLAC file
audio = AudioSegment.from_file("./audio/bal_train/--cB2ZVjpnA.flac", format="flac")

# Resample the audio to 16kHz
resampled_audio = audio.set_frame_rate(16000)

# Export the resampled audio
resampled_audio.export("test.flac", format="flac")

<_io.BufferedRandom name='test.flac'>

In [4]:
# Load your audio file
audio = AudioSegment.from_file("test.flac")

# Get sampling rate
sampling_rate = audio.frame_rate

# Get bitrate
bitrate = audio.frame_width * audio.frame_rate * 8

print(f"Frame Width: {audio.frame_width} (I think it's in Bytes)")
print(f"Sampling Rate: {sampling_rate/1000} kHz")
print(f"Bitrate: {bitrate/1000} kbps")

Frame Width: 8 (I think it's in Bytes)
Sampling Rate: 16.0 kHz
Bitrate: 1024.0 kbps


In [29]:
# Load your audio file
audio = AudioSegment.from_file("flac_16kHz/-i-9C48tduw.flac")

# Get sampling rate
sampling_rate = audio.frame_rate

# Get bitrate
bitrate = audio.frame_width * audio.frame_rate * 8

print(f"Frame Width: {audio.frame_width} (I think it's in Bytes)")
print(f"Sampling Rate: {sampling_rate/1000} kHz")
print(f"Bitrate: {bitrate/1000} kbps")

Frame Width: 8 (I think it's in Bytes)
Sampling Rate: 16.0 kHz
Bitrate: 1024.0 kbps


## LTU Open-QA

In [2]:
import json
from tqdm import tqdm

In [3]:
with open("../ltu-openasqa/openasqa_10.3M_v2.json") as f:
    all_data = json.load(f)

In [5]:
with open("../ltu-openasqa/openasqa_10.3M_v2.thai.json") as f:
    all_data_thai = json.load(f)

In [6]:
len(all_data), len(all_data_thai)

(10324230, 10324230)

In [7]:
audioset_data = []
for x in tqdm(all_data):
    if "audioset/dave_version" in x['audio_id']: 
        audioset_data.append(x)

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:05<00:00, 1752924.53it/s]


In [8]:
len(audioset_data)

4309357

In [9]:
audioset_data_thai = []
for x in tqdm(all_data_thai):
    if "audioset/dave_version" in x['audio_id']: 
        audioset_data_thai.append(x)

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:04<00:00, 2115575.17it/s]


In [25]:
flac_names = set()
for x in tqdm(audioset_data):
    flac_name = x['audio_id'].split("/")[-1]
    flac_names.add(flac_name)

100%|█████████████████████████████████████████████████████████████████████████████████| 4309357/4309357 [00:05<00:00, 800883.27it/s]


In [26]:
len(flac_names)

523258

In [27]:
flac_names = list(flac_names)

In [28]:
with open("ltu-audioset-ids.txt", "w") as f:
    f.write("\n".join(flac_names))

# Write a run script

In [34]:
def write(path, start_id, end_id):
    # [start_id, end_id)
    text = ""
    text += "#!/bin/bash\n"
    text += "#$ -S /bin/bash\n\n"
    text += "export HF_DATASETS_CACHE=/cache/.cache/huggingface/datasets\n"
    text += "export HF_HOME=/cache/.cache/huggingface\n"
    text += "export HF_HUB_CACHE=/cache/.cache/huggingface/hub\n\n"
    for i in range(start_id, end_id):
        text += "python download_filter_resample.py unbal_train{:03}\n".format(i)
    # print(text)
    with open(path, "w") as f:
        f.write(text)

In [35]:
write("./run_download_unbal_0.sh", 0, 100)

In [36]:
write("./run_download_unbal_1.sh", 100, 200)

In [37]:
write("./run_download_unbal_2.sh", 200, 300)

In [38]:
write("./run_download_unbal_3.sh", 300, 400)

In [39]:
write("./run_download_unbal_4.sh", 400, 500)

In [40]:
write("./run_download_unbal_5.sh", 500, 600)

In [41]:
write("./run_download_unbal_6.sh", 600, 700)

In [42]:
write("./run_download_unbal_7.sh", 700, 800)

In [ ]:
write("./run_download_unbal_8.sh", 800, 870)

# Prepare training data

In [75]:
# run those cells above
import os

In [76]:
len(all_data), len(all_data_thai)

(10324230, 10324230)

In [77]:
all_data[100]

{'instruction': 'What acoustic feature characterizes the sound of breathing in the audio clip?',
 'input': '300 believers',
 'audio_id': '/data/sls/audioset/dave_version/audio/-qb8GQR7IqM.flac',
 'dataset': 'as_strong_train',
 'task': 'open-ended question',
 'output': 'The sound of breathing is low-frequency and rhythmic.'}

In [78]:
all_data_thai[100]

{'instruction': 'คุณลักษณะทางเสียงใดที่ทำให้เกิดเสียงหายใจในคลิปเสียง',
 'input': 'ผู้ศรัทธา 300 คน',
 'output': 'เสียงหายใจเป็นเสียงความถี่ต่ำและเป็นจังหวะ',
 'audio_id': '/data/sls/audioset/dave_version/audio/-qb8GQR7IqM.flac',
 'dataset': 'as_strong_train',
 'task': 'open-ended question'}

In [79]:
audioset_data = []
for x in tqdm(all_data):
    if "audioset" in x['dataset']: 
        audioset_data.append(x)

audioset_data_thai = []
for x in tqdm(all_data_thai):
    if "audioset" in x['dataset']: 
        audioset_data_thai.append(x)

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:03<00:00, 2776713.21it/s]


In [80]:
len(audioset_data), len(audioset_data_thai)

(1429437, 1429437)

In [81]:
audioset_data[0]

{'audio_id': '/data/sls/audioset/dave_version/audio/KEOHeXbwFEs.flac',
 'input': "I'm going to do a quick run through of my beautiful, colorful sheep.",
 'instruction': "What's the implied meaning of the audio clip?",
 'output': "From the recording, it can be inferred that the speaker is introducing their sheep, likely to an audience or for a personal project. The background sound of sheep bleating suggests that the speaker is in close proximity to the animals. The speech and background sound are related as the speaker seems to be showcasing their sheep to the audience or possibly documenting their work with the animals. This type of speech and background sound would likely be heard in an agricultural setting or at a livestock show. Overall, the recording seems to capture a moment of pride and enthusiasm for the speaker's sheep.",
 'task': 'open-ended question',
 'dataset': 'audioset'}

In [82]:
audioset_data_thai[0]

{'instruction': 'ความหมายโดยนัยของคลิปเสียงคืออะไร?',
 'input': 'ฉันจะวิ่งดูแกะแสนสวยของฉันอย่างรวดเร็ว',
 'output': 'จากการบันทึก สามารถอนุมานได้ว่าผู้พูดกำลังแนะนำแกะของตน ซึ่งน่าจะเกิดขึ้นกับผู้ฟังหรือสำหรับโครงการส่วนตัว เสียงพื้นหลังของเสียงร้องของแกะบ่งบอกว่าผู้พูดอยู่ใกล้สัตว์มาก คำพูดและเสียงพื้นหลังมีความสัมพันธ์กันเนื่องจากผู้พูดดูเหมือนจะจัดแสดงแกะให้ผู้ชมเห็นหรืออาจบันทึกการทำงานร่วมกับสัตว์ต่างๆ เสียงพูดและเสียงพื้นหลังประเภทนี้มักจะได้ยินในสถานที่เกษตรกรรมหรือในการแสดงปศุสัตว์ โดยรวมแล้ว การบันทึกดังกล่าวดูเหมือนจะจับภาพช่วงเวลาแห่งความภาคภูมิใจและความกระตือรือร้นต่อแกะของผู้บรรยาย',
 'audio_id': '/data/sls/audioset/dave_version/audio/KEOHeXbwFEs.flac',
 'dataset': 'audioset',
 'task': 'open-ended question'}

In [83]:
en_examples = []
count0, count1 = 0, 0
for x in tqdm(audioset_data):
    file_name = x['audio_id'].split("/")[-1]
    path = f"/dataset/audio-data/audioset/flac_16kHz/{file_name}"
    if os.path.isfile(path) == False:
        count0 += 1
        continue
    else:
        count1 += 1
    en_examples.append({
        'path': path,
        'task': "QA",
        'Q': x['instruction'],
        'text': x['output'],
    })

100%|█████████████████████████████████████████████████████████████████████████████████| 1429437/1429437 [00:08<00:00, 159885.54it/s]


In [84]:
count0, count1

(26378, 1403059)

In [85]:
en_examples[20000]

{'path': '/dataset/audio-data/audioset/flac_16kHz/ahvEBUwSslU.flac',
 'task': 'QA',
 'Q': 'Where is this scenario likely taking place?',
 'text': 'This scenario is likely taking place at a birthday party or event.'}

In [86]:
with open("./audioset_train.en.json", "w") as f:
    json.dump(en_examples, f, indent=4)

In [87]:
th_examples = []
count0, count1 = 0, 0
for x in tqdm(audioset_data_thai):
    file_name = x['audio_id'].split("/")[-1]
    path = f"/dataset/audio-data/audioset/flac_16kHz/{file_name}"
    if os.path.isfile(path) == False:
        count0 += 1
        continue
    else:
        count1 += 1
    th_examples.append({
        'path': path,
        'task': "QA",
        'Q': x['instruction'],
        'text': x['output'],
    })

100%|█████████████████████████████████████████████████████████████████████████████████| 1429437/1429437 [00:09<00:00, 155971.79it/s]


In [88]:
count0, count1

(26378, 1403059)

In [89]:
th_examples[:10]

[{'path': '/dataset/audio-data/audioset/flac_16kHz/KEOHeXbwFEs.flac',
  'task': 'QA',
  'Q': 'ความหมายโดยนัยของคลิปเสียงคืออะไร?',
  'text': 'จากการบันทึก สามารถอนุมานได้ว่าผู้พูดกำลังแนะนำแกะของตน ซึ่งน่าจะเกิดขึ้นกับผู้ฟังหรือสำหรับโครงการส่วนตัว เสียงพื้นหลังของเสียงร้องของแกะบ่งบอกว่าผู้พูดอยู่ใกล้สัตว์มาก คำพูดและเสียงพื้นหลังมีความสัมพันธ์กันเนื่องจากผู้พูดดูเหมือนจะจัดแสดงแกะให้ผู้ชมเห็นหรืออาจบันทึกการทำงานร่วมกับสัตว์ต่างๆ เสียงพูดและเสียงพื้นหลังประเภทนี้มักจะได้ยินในสถานที่เกษตรกรรมหรือในการแสดงปศุสัตว์ โดยรวมแล้ว การบันทึกดังกล่าวดูเหมือนจะจับภาพช่วงเวลาแห่งความภาคภูมิใจและความกระตือรือร้นต่อแกะของผู้บรรยาย'},
 {'path': '/dataset/audio-data/audioset/flac_16kHz/UExUU9LEBmQ.flac',
  'task': 'QA',
  'Q': 'การรวมเสียงของเครื่องดนตรีควบคู่ไปกับเนื้อหาคำพูดมีความสำคัญอย่างไร และสิ่งนี้จะส่งผลต่อการรับรู้ของผู้ฟังอย่างไร',
  'text': 'การใช้เครื่องดนตรีทำให้เกิดการตอบสนองทางอารมณ์ในผู้ฟัง ทำให้คำพูดมีผลกระทบและน่าจดจำมากขึ้น'},
 {'path': '/dataset/audio-data/audioset/flac_16kHz/_Mu

In [91]:
with open("./audioset_train.th.json", "w") as f:
    json.dump(th_examples, f, indent=4, ensure_ascii=False)

# AS-Strong & AudioSet (Audio)

In [57]:
task_count = {}
for x in tqdm(all_data):
    if 'audioset' in x['audio_id']:
        if 'audiocaps' in x['dataset'] or 'as_strong' in x['dataset'] or 'audioset' == x['dataset']:
            continue
        t = (x['dataset'], x['task'])
        if t not in task_count:
            task_count[t] = 1
        else:
            task_count[t] += 1

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:06<00:00, 1711263.95it/s]


In [58]:
task_count

{('as_20k', 'open-ended question'): 183590,
 ('as_2m', 'cla_label_des'): 500139,
 ('as_20k', 'cla_label'): 18691,
 ('as_20k', 'cla_label_des'): 18691}

In [59]:
sum(v for v in task_count.values())

721111

In [92]:
as_strong_data = []
count = 0
for x in tqdm(all_data):
    if 'as_strong' not in x['dataset']:
        continue
    file_name = x['audio_id'].split("/")[-1]
    path = f"/dataset/audio-data/audioset/flac_16kHz/{file_name}"
    if os.path.isfile(path) == False:
        count += 1
        continue
    as_strong_data.append({
        'path': path, 
        'task': "QA",
        'Q': x['instruction'],
        'text': x['output'],
    })

100%|███████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:12<00:00, 854946.52it/s]


In [94]:
len(as_strong_data), count

(1555455, 28682)

In [96]:
with open("./as_strong_train.en.json", "w") as f:
    json.dump(as_strong_data, f, indent=4)

In [97]:
as_20k_data = []
count = 0
for x in tqdm(all_data):
    if x['dataset'] == "as_20k" and x['task'] == "open-ended question":    
        file_name = x['audio_id'].split("/")[-1]
        path = f"/dataset/audio-data/audioset/flac_16kHz/{file_name}"
        if os.path.isfile(path) == False:
            count += 1
            continue
        as_20k_data.append({
            'path': path, 
            'task': "QA",
            'Q': x['instruction'],
            'text': x['output'],
        })

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:05<00:00, 2031249.80it/s]


In [98]:
len(as_20k_data), count

(180085, 3505)

In [99]:
with open("./as_20k_train.en.json", "w") as f:
    json.dump(as_20k_data, f, indent=4)

In [100]:
as_20k_data_thai = []
count = 0
for x in tqdm(all_data_thai):
    if x['dataset'] == "as_20k" and x['task'] == "open-ended question":    
        file_name = x['audio_id'].split("/")[-1]
        path = f"/dataset/audio-data/audioset/flac_16kHz/{file_name}"
        if os.path.isfile(path) == False:
            count += 1
            continue
        as_20k_data_thai.append({
            'path': path, 
            'task': "QA",
            'Q': x['instruction'],
            'text': x['output'],
        })

100%|██████████████████████████████████████████████████████████████████████████████| 10324230/10324230 [00:04<00:00, 2101246.73it/s]


In [101]:
len(as_20k_data_thai), count

(180085, 3505)

In [102]:
as_20k_data_thai[0]

{'path': '/dataset/audio-data/audioset/flac_16kHz/UhXKXjckLyY.flac',
 'task': 'QA',
 'Q': 'คลิปเสียงนี้สื่อถึงอารมณ์หรือบรรยากาศใด',
 'text': 'คลิปเสียงสื่อถึงอารมณ์หรือบรรยากาศที่น่าเศร้าหรือน่าสะเทือนใจเนื่องจากมีเสียงครวญครางและร้องไห้ของมนุษย์'}

In [103]:
with open("./as_20k_train.th.json", "w") as f:
    json.dump(as_20k_data_thai, f, indent=4, ensure_ascii=False)